# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-ef712bc361-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA seasons explorative search") 

Consider the following exploratory information need:

>  investigate the world of basketball. Compare french teams and US teams, country of citizenship of NBA players, and teams which won NBA seasons.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q41421`| Michael Jordan     | node |
| `wd:Q25369`| Kobe Bryant     | node |





Also consider

```
wd:Q41421 ?p ?obj .
```

is the BGP to retrieve all **properties of Michael Jordan**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all french basketball teams born before 1930 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Return all the NBA season (the result set must be a list of couples NBA season IRI, label)

6. Consider only the basketball teams which Michael Jordan and Kobe Bryant played. Return the number of NBA seasons won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team even if they never won a season).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, label and #players)

8. Identify the BGP for the NBA Defensive Player of the Year Award (see https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award)

9. Consider the players who won the NBA Defensive Player of the Year Award. Return for these players the number of NBA Defensive Player of the Year Award won in their career (the result set must be triples of player IRI, label and #victories)


## Task 1
Identify the BGP for basketball player.

In [84]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q41421 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1048'), ('name', 'NCL ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p'

Final query for this task

In [6]:
queryString = """
SELECT DISTINCT ?job ?jobName
WHERE {
   # using Profession property
   wd:Q41421 wdt:P106 ?job .
   
   # get the label
   ?job sc:name ?jobName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('job', 'http://www.wikidata.org/entity/Q131524'), ('jobName', 'entrepreneur')]
[('job', 'http://www.wikidata.org/entity/Q18814623'), ('jobName', 'autobiographer')]
[('job', 'http://www.wikidata.org/entity/Q33999'), ('jobName', 'actor')]
[('job', 'http://www.wikidata.org/entity/Q3665646'), ('jobName', 'basketball player')]
[('job', 'http://www.wikidata.org/entity/Q10871364'), ('jobName', 'baseball player')]
[('job', 'http://www.wikidata.org/entity/Q17221'), ('jobName', 'spokesperson')]
6


## Task 2
Identify the BGP for basketball.

In [7]:
# From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]

queryString = """
SELECT DISTINCT ?sport ?sportName
WHERE {
   # using Profession property
   wd:Q41421 wdt:P641 ?sport .
   
   # get the label
   ?sport sc:name ?sportName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q5369'), ('sportName', 'baseball')]
[('sport', 'http://www.wikidata.org/entity/Q5372'), ('sportName', 'basketball')]
2


Final query for this task

In [8]:
queryString = """
SELECT DISTINCT ?sport ?sportName
WHERE {
   # using Profession property
   wd:Q41421 wdt:P641 ?sport .
   
   # get the label
   ?sport sc:name ?sportName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q5369'), ('sportName', 'baseball')]
[('sport', 'http://www.wikidata.org/entity/Q5372'), ('sportName', 'basketball')]
2


## Task 3
Identify the BGP for basketball team.

In [9]:

queryString = """
SELECT DISTINCT ?job ?jobName
WHERE {
   # using Profession property
   wd:Q41421 wdt:P106 ?job .
   
   
   # get the label
   ?job sc:name ?jobName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('job', 'http://www.wikidata.org/entity/Q131524'), ('jobName', 'entrepreneur')]
[('job', 'http://www.wikidata.org/entity/Q18814623'), ('jobName', 'autobiographer')]
[('job', 'http://www.wikidata.org/entity/Q33999'), ('jobName', 'actor')]
[('job', 'http://www.wikidata.org/entity/Q3665646'), ('jobName', 'basketball player')]
[('job', 'http://www.wikidata.org/entity/Q10871364'), ('jobName', 'baseball player')]
[('job', 'http://www.wikidata.org/entity/Q17221'), ('jobName', 'spokesperson')]
6


In [11]:
#[('job', 'http://www.wikidata.org/entity/Q3665646'), ('jobName', 'basketball player')]

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # Finding basketball player properties
   wd:Q3665646 ?p ?o .
   
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q18536342'), ('oname', 'competitive player')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P4557'), ('oname', 'Indiana Basketball Hall of Fame ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('oname', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pname', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2685'), ('oname', 'Basketball Reference NBA player ID'

Final query for this task

In [13]:
# From results: [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'),
#('o', 'http://www.wikidata.org/entity/Q13393265'),('oname', 'basketball team')]


queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # Using part of property
   wd:Q3665646 wdt:P361 ?o .
   
   
   # get the label
   #?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('o', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
1


## Task 4
Return all french basketball teams born before 1930 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [16]:
#From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]

queryString = """
SELECT DISTINCT ?team ?teamName
WHERE {
   # Finding basketball team properties
   wd:Q41421 wdt:P54 ?team.
   
   
   # get the label
   ?team sc:name ?teamName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('team', 'http://www.wikidata.org/entity/Q128109'), ('teamName', 'Chicago Bulls')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('teamName', 'Washington Wizards')]
[('team', 'http://www.wikidata.org/entity/Q2747877'), ('teamName', 'Birmingham Barons')]
[('team', 'http://www.wikidata.org/entity/Q3476102'), ('teamName', 'Scottsdale Scorpions')]
[('team', 'http://www.wikidata.org/entity/Q4412061'), ('teamName', "North Carolina Tar Heels men's basketball")]
5


In [20]:
#From results: [('team', 'http://www.wikidata.org/entity/Q128109'), ('teamName', 'Chicago Bulls')]


queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # Finding sports club properties
   wd:Q128109 ?p ?o.
   
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P859'), ('pname', 'sponsor'), ('o', 'http://www.wikidata.org/entity/Q174769'), ('oname', 'United Airlines')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pname', 'home venue'), ('o', 'http://www.wikidata.org/entity/Q639975'), ('oname', 'United Center')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q1297'), ('oname', 'Chicago')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league'), ('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname'

In [14]:
# [('o', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # Finding basketball team properties
   wd:Q13393265 ?p ?o.
   
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q51747567'), ('oname', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3665646'), ('oname', 'basketball player')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q5137571'), ('oname', 'basketball coach')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q847017'), ('oname', 'sports club')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main temp

In [15]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'),
#('o', 'http://www.wikidata.org/entity/Q847017'),('oname', 'sports club')]


queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # Finding sports club properties
   wd:Q847017 ?p ?o.
   
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pname', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q51747567'), ('oname', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oname', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4438121'), ('oname', 'sports organization')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q19467997'), ('oname', 'Template:Infobox athletics club')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q988108'), ('oname', 'club')

In [24]:
#[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), 
#('o', 'http://www.wikidata.org/entity/Q4438121'), ('oname', 'sports organization')]

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # Finding sports organization properties
   wd:Q4438121 ?p ?o.
   
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q349'), ('oname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q43229'), ('oname', 'organization')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q4088044'), ('oname', 'Category:Sports organizations')]
3


In [59]:
#[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), 
#('o', 'http://www.wikidata.org/entity/Q43229'), ('oname', 'organization')]

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # Finding organization properties
   wd:Q43229 ?p ?o.
   
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3500'), ('oname', 'Ringgold ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P213'), ('oname', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1705'), ('oname', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3549'), ('oname', 'Australian Company Number')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1278'), ('oname', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1320'), ('oname',

In [52]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'),
#('o', 'http://www.wikidata.org/entity/P17'), ('oname', 'country')]

#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('o', 'http://www.wikidata.org/entity/P571'), ('oname', 'inception')]

# [('o', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]

queryString = """
SELECT DISTINCT ?team ?teamName ?countryName ?inception
WHERE {
   
   # set team as an instance of a basketball team
   ?team wdt:P31 wd:Q13393265 ;
         wdt:P17 ?country ;
         wdt:P571 ?inception .
   
   
   # get the label
   ?team sc:name ?teamName .
   ?country sc:name ?countryName.
   
   #filtering to get all French teams with their date of inception
   
   FILTER ( REGEX (?countryName, "^[Ff]rance$") )
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q795707'), ('teamName', 'BCM Gravelines'), ('countryName', 'France'), ('inception', '1984-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q1075934'), ('teamName', 'Cholet Basket'), ('countryName', 'France'), ('inception', '1975-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q2892731'), ('teamName', 'Paris Université Club'), ('countryName', 'France'), ('inception', '1906-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q404421'), ('teamName', 'STB Le Havre'), ('countryName', 'France'), ('inception', '1924-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q2887227'), ('teamName', 'Basket Landes'), ('countryName', 'France'), ('inception', '2003-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q40063'), ('teamName', 'Le Mans Sarthe Basket'), ('countryName', 'France'), ('inception', '1939-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q2868091'), ('teamName', 'AS Mon

Final query for this task

In [51]:

queryString = """
SELECT DISTINCT ?team ?teamName ( YEAR(?inception) as ?Inception ) 
WHERE {
   
   # Finding basketball team properties
   ?team wdt:P31 wd:Q13393265 ;
         wdt:P17 ?country ;
         wdt:P571 ?inception .
   
   
   # get the label
   ?team sc:name ?teamName .
   ?country sc:name ?countryName.
   
   FILTER ( REGEX (?countryName, "^[Ff]rance$") )
   FILTER ( YEAR(?inception) < 1930 )
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('team', 'http://www.wikidata.org/entity/Q2892731'), ('teamName', 'Paris Université Club'), ('Inception', '1906')]
[('team', 'http://www.wikidata.org/entity/Q404421'), ('teamName', 'STB Le Havre'), ('Inception', '1924')]
[('team', 'http://www.wikidata.org/entity/Q2868091'), ('teamName', 'AS Monaco Basket'), ('Inception', '1924')]
[('team', 'http://www.wikidata.org/entity/Q1630650'), ('teamName', 'JL Bourg-en-Bresse'), ('Inception', '1910')]
[('team', 'http://www.wikidata.org/entity/Q2868120'), ('teamName', 'Union Tours Basket Métropole'), ('Inception', '1925')]
[('team', 'http://www.wikidata.org/entity/Q2931972'), ('teamName', 'Basket CRO Lyon'), ('Inception', '1927')]
[('team', 'http://www.wikidata.org/entity/Q2887216'), ('teamName', 'BC Orchies'), ('Inception', '1922')]
[('team', 'http://www.wikidata.org/entity/Q3592481'), ('teamName', 'Étoile de Charleville-Mézières'), ('Inception', '1921')]
[('team', 'http://www.wikidata.org/entity/Q638938'), ('teamName', 'Paris Basket Rac

## Task 5
Return all the NBA season (the result set must be a list of couples NBA season IRI, label)

In [57]:
# From Task 4 (properties for [('team', 'http://www.wikidata.org/entity/Q128109'), ('teamName', 'Chicago Bulls')] :

#[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league'), 
#('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # Finding National Basketball Association properties
   wd:Q155223 ?p ?o.
   
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
   
   #FILTER (REGEX(?oname , ".*season.*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18536323'), ('oname', 'basketball league')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15986354'), ('oname', 'basketball association')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q238240'), ('oname', 'Eastern Conference')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pname', 'owner of'), ('o', 'http://www.wikidata.org/entity/Q2455323'), ('oname', 'NBA TV')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q810343'), ('oname', 'Basketball Association of America')]
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('pname', 'season ends'), ('o', 'http://www.wikidata.org/entity/Q118'), ('oname', 'April')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [94]:
queryString = """
SELECT DISTINCT  ?inst ?instname
WHERE {
   # Finding National Basketball Association instances
   ?inst wdt:P31 wd:Q155223 .
   
   
   # get the label
   ?inst sc:name ?instname .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [96]:
#('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]


queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # trying to find related property for NBA as an object
    ?s ?p wd:Q155223 .
   
   # get the label
   ?p sc:name ?pname.
  
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1591'), ('pname', 'defendant')]
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('pname', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P264'), ('pname', 'record label')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direc

In [98]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]

queryString = """
SELECT DISTINCT ?season ?seasonName
WHERE {
   # Finding National Basketball Association instances
    ?season wdt:P3450 wd:Q155223 .
   
   # get the label
   ?season sc:name ?seasonName.
  
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('season', 'http://www.wikidata.org/entity/Q600389'), ('seasonName', '1990–91 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q2264531'), ('seasonName', '1967–68 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q241834'), ('seasonName', '1989–90 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q602553'), ('seasonName', '1969–70 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q2294127'), ('seasonName', '1971–72 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q113619'), ('seasonName', '2012–13 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q16191100'), ('seasonName', '2014-15 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q19862716'), ('seasonName', '2015–16 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q631240'), ('seasonName', '1979–80 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q1142320'), ('seasonName', '2005–06 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q281094'), 

Final query for this task

In [2]:

queryString = """
SELECT DISTINCT ?season ?seasonName
WHERE {
   # Finding National Basketball Association instances
    ?season wdt:P3450 wd:Q155223 .
   
   # get the label
   ?season sc:name ?seasonName.
  
   
}

"""

print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q600389'), ('seasonName', '1990–91 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q2264531'), ('seasonName', '1967–68 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q241834'), ('seasonName', '1989–90 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q602553'), ('seasonName', '1969–70 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q2294127'), ('seasonName', '1971–72 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q113619'), ('seasonName', '2012–13 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q16191100'), ('seasonName', '2014-15 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q19862716'), ('seasonName', '2015–16 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q631240'), ('seasonName', '1979–80 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q1142320'), ('seasonName', '2005–06 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q281094'), 

## Task 6
Consider only the basketball teams which Michael Jordan and Kobe Bryant played. Return the number of NBA seasons won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team even if they never won a season).

In [88]:
#From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]

queryString = """
SELECT DISTINCT ?player ?playerName ?team ?teamName
WHERE {
   
   VALUES ?player { wd:Q41421 wd:Q25369 }
   # using member of sports team property
   ?player  wdt:P54 ?team .
   
   
   # get the label
   ?player sc:name ?playerName .
   ?team sc:name ?teamName .
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)


Results
[('player', 'http://www.wikidata.org/entity/Q25369'), ('playerName', 'Kobe Bryant'), ('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('playerName', 'Michael Jordan'), ('team', 'http://www.wikidata.org/entity/Q128109'), ('teamName', 'Chicago Bulls')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('playerName', 'Michael Jordan'), ('team', 'http://www.wikidata.org/entity/Q169165'), ('teamName', 'Washington Wizards')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('playerName', 'Michael Jordan'), ('team', 'http://www.wikidata.org/entity/Q2747877'), ('teamName', 'Birmingham Barons')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('playerName', 'Michael Jordan'), ('team', 'http://www.wikidata.org/entity/Q3476102'), ('teamName', 'Scottsdale Scorpions')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('playerName', 'Michael Jordan'), ('team', 'http://www.wikidata

In [100]:
#From Task 5 as an example of a season:
#[('season', 'http://www.wikidata.org/entity/Q600389'),('seasonName', '1990–91 NBA season')]

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # Finding an NBA season properties
   wd:Q600389 ?p ?o.
   
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)





Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8172698'), ('oname', 'Category:1990–91 NBA season')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pname', 'time period'), ('o', 'http://www.wikidata.org/entity/Q53389240'), ('oname', '1990-1991 one-year-period')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q27020041'), ('oname', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pname', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q46343203'), ('oname', "men's basketball")]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www

In [104]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'),
#('o', 'http://www.wikidata.org/entity/Q128109'), ('oname', 'Chicago Bulls')]

queryString = """
SELECT DISTINCT ?seasonName ?teamName
WHERE {

   #All teams Jordan and Kobe played in
   VALUES ?team { wd:Q121783 wd:Q128109 wd:Q169165 wd:Q2747877 wd:Q3476102 wd:Q4412061 } 
   # all NBA seasons
   ?season wdt:P3450 wd:Q155223 .
   
   ?season wdt:P1346 ?team .
   
   
   # get the label
   ?season sc:name ?seasonName .
   ?team sc:name ?teamName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('seasonName', '1990–91 NBA season'), ('teamName', 'Chicago Bulls')]
[('seasonName', '1971–72 NBA season'), ('teamName', 'Los Angeles Lakers')]
[('seasonName', '1979–80 NBA season'), ('teamName', 'Los Angeles Lakers')]
[('seasonName', '1949–50 NBA season'), ('teamName', 'Los Angeles Lakers')]
[('seasonName', '1977–78 NBA season'), ('teamName', 'Washington Wizards')]
[('seasonName', '2001–02 NBA season'), ('teamName', 'Los Angeles Lakers')]
[('seasonName', '1999–2000 NBA season'), ('teamName', 'Los Angeles Lakers')]
[('seasonName', '2000–01 NBA season'), ('teamName', 'Los Angeles Lakers')]
[('seasonName', '1992–93 NBA season'), ('teamName', 'Chicago Bulls')]
[('seasonName', '1995–96 NBA season'), ('teamName', 'Chicago Bulls')]
[('seasonName', '1996–97 NBA season'), ('teamName', 'Chicago Bulls')]
[('seasonName', '1997–98 NBA season'), ('teamName', 'Chicago Bulls')]
[('seasonName', '1952–53 NBA season'), ('teamName', 'Los Angeles Lakers')]
[('seasonName', '2019–20 NBA season'), (

In [ ]:

queryString = """
SELECT DISTINCT ?team ?teamName (COUNT(?season) as ?winner)
WHERE {

 
   #All teams Jordan and Kobe played in
   VALUES ?team { wd:Q121783 wd:Q128109 wd:Q169165 wd:Q2747877 wd:Q3476102 wd:Q4412061 } 

   # all NBA seasons
   ?season wdt:P3450 wd:Q155223 .
   
   # all seasons with a winner from ?team
   ?season wdt:P1346 ?team .
   
   # get the label
   ?season sc:name ?seasonName .
   ?team sc:name ?teamName .
}
GROUP BY (?team) (?teamName)
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers'), ('winner', '17')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('teamName', 'Washington Wizards'), ('winner', '1')]
[('team', 'http://www.wikidata.org/entity/Q128109'), ('teamName', 'Chicago Bulls'), ('winner', '6')]
3


In [51]:
#From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]
#('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]
#[('x', 'http://www.wikidata.org/prop/direct/P3450'), ('xname', 'sports season of league or competition')]
#('xname', '1990–91 NBA season'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]

queryString = """
SELECT DISTINCT  ?team ?teamName (count (?team) as ?winner)
WHERE {
    VALUES ?player {wd:Q41421  wd:Q25369}
   
    ?player wdt:P54 ?team .
    
    optional{
    ?season wdt:P3450 wd:Q155223.
    ?season wdt:P1346 ?team.
    }

    ?team sc:name ?teamName.
 
   
}
GROUP BY (?team) (?teamName)

LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers'), ('winner', '17')]
[('team', 'http://www.wikidata.org/entity/Q3476102'), ('teamName', 'Scottsdale Scorpions'), ('winner', '1')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('teamName', 'Washington Wizards'), ('winner', '1')]
[('team', 'http://www.wikidata.org/entity/Q128109'), ('teamName', 'Chicago Bulls'), ('winner', '6')]
[('team', 'http://www.wikidata.org/entity/Q2747877'), ('teamName', 'Birmingham Barons'), ('winner', '1')]
[('team', 'http://www.wikidata.org/entity/Q4412061'), ('teamName', "North Carolina Tar Heels men's basketball"), ('winner', '1')]
6


Final query for this task

In [106]:


queryString = """
SELECT DISTINCT ?team ?teamName (COUNT(?season) as ?winner)
WHERE {

 
   #All teams Jordan and Kobe played in
   VALUES ?team { wd:Q121783 wd:Q128109 wd:Q169165 wd:Q2747877 wd:Q3476102 wd:Q4412061 } 

   {
   # all NBA seasons
   ?season wdt:P3450 wd:Q155223 .
   
   # all seasons with a winner from ?team
   ?season wdt:P1346 ?team .
   
     # get the label
     ?season sc:name ?seasonName .
     ?team sc:name ?teamName .
   }
   UNION
   {
   
    ?team ?p ?o .
   
     # get the label
     ?team sc:name ?teamName .
   
   }
 
}
GROUP BY (?team) (?teamName)
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers'), ('winner', '17')]
[('team', 'http://www.wikidata.org/entity/Q3476102'), ('teamName', 'Scottsdale Scorpions'), ('winner', '0')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('teamName', 'Washington Wizards'), ('winner', '1')]
[('team', 'http://www.wikidata.org/entity/Q128109'), ('teamName', 'Chicago Bulls'), ('winner', '6')]
[('team', 'http://www.wikidata.org/entity/Q2747877'), ('teamName', 'Birmingham Barons'), ('winner', '0')]
[('team', 'http://www.wikidata.org/entity/Q4412061'), ('teamName', "North Carolina Tar Heels men's basketball"), ('winner', '0')]
6


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, label and #players)

In [134]:
#('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]


queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # trying to find related property for NBA as an object
    ?s ?p wd:Q155223 .
   
   # get the label
   ?p sc:name ?pname.
  
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1591'), ('pname', 'defendant')]
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('pname', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P264'), ('pname', 'record label')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direc

In [137]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league')]

## [('o', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
##('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]


queryString = """
SELECT DISTINCT ?team ?teamName
WHERE {
   # all NBA teams
    ?team wdt:P31 wd:Q13393265 ;
          wdt:P118 wd:Q155223 .
   
   # get the label
   ?team sc:name ?teamName.
  
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q131364'), ('teamName', 'New York Knicks')]
[('team', 'http://www.wikidata.org/entity/Q162954'), ('teamName', 'Denver Nuggets')]
[('team', 'http://www.wikidata.org/entity/Q167253'), ('teamName', 'Portland Trail Blazers')]
[('team', 'http://www.wikidata.org/entity/Q180950'), ('teamName', 'Oklahoma City Thunder')]
[('team', 'http://www.wikidata.org/entity/Q976396'), ('teamName', 'Los Angeles Clippers')]
[('team', 'http://www.wikidata.org/entity/Q166105'), ('teamName', 'Sacramento Kings')]
[('team', 'http://www.wikidata.org/entity/Q170649'), ('teamName', 'Utah Jazz')]
[('team', 'http://www.wikidata.org/entity/Q138089'), ('teamName', 'Philadelphia 76ers')]
[('team', 'http://www.wikidata.org/entity/Q161345'), ('teamName', 'Houston Rockets')]
[('team', 'http://www.wikidata.org/entity/Q164177'), ('teamName', 'Phoenix Suns')]
[('team', 'http://www.wikidata.org/entity/Q172339'), ('teamName', 'New Orleans Pelicans')]
[('team', 'http://www.wikidat

In [141]:
# From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]
#[('job', 'http://www.wikidata.org/entity/Q3665646'), ('jobName', 'basketball player')]

queryString = """
SELECT DISTINCT ?playerName ?teamName
WHERE {
   # all NBA teams
    ?team wdt:P31 wd:Q13393265 ;
          wdt:P118 wd:Q155223 .
   
   #all players played for an NBA team
    ?player wdt:P31 wd:Q5 ;
            wdt:P106 wd:Q3665646 ;
            wdt:P54 ?team .
   
   # get the label
   ?team sc:name ?teamName.
   ?player sc:name ?playerName .
  
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)




Results
[('playerName', 'Bill Bradley'), ('teamName', 'New York Knicks')]
[('playerName', 'Stephon Marbury'), ('teamName', 'New York Knicks')]
[('playerName', 'DerMarr Johnson'), ('teamName', 'New York Knicks')]
[('playerName', 'Jason Kidd'), ('teamName', 'New York Knicks')]
[('playerName', 'Bruno Šundov'), ('teamName', 'New York Knicks')]
[('playerName', 'Gene Shue'), ('teamName', 'New York Knicks')]
[('playerName', 'Ernie Grunfeld'), ('teamName', 'New York Knicks')]
[('playerName', 'Emmanuel Mudiay'), ('teamName', 'New York Knicks')]
[('playerName', 'James Bailey'), ('teamName', 'New York Knicks')]
[('playerName', 'Danilo Gallinari'), ('teamName', 'New York Knicks')]
[('playerName', 'Sergio Rodríguez'), ('teamName', 'New York Knicks')]
[('playerName', 'Carmelo Anthony'), ('teamName', 'New York Knicks')]
[('playerName', 'Scott Brooks'), ('teamName', 'New York Knicks')]
[('playerName', 'Paul Westphal'), ('teamName', 'New York Knicks')]
[('playerName', 'Patrick Ewing'), ('teamName', 'Ne

In [145]:
queryString = """
SELECT DISTINCT ?playerName ?teamName ?countryName
WHERE {
   # all NBA teams
    ?team wdt:P31 wd:Q13393265 ;
          wdt:P118 wd:Q155223 .
   
   #all players played for an NBA team with their nationality
    ?player wdt:P31 wd:Q5 ;
            wdt:P106 wd:Q3665646 ;
            wdt:P54 ?team ;
            wdt:P27 ?country .
   
   # get the label
   ?team sc:name ?teamName.
   ?player sc:name ?playerName .
   ?country sc:name ?countryName .
  
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('playerName', 'Ime Udoka'), ('teamName', 'New York Knicks'), ('countryName', 'Nigeria')]
[('playerName', 'Cheikh Samb'), ('teamName', 'New York Knicks'), ('countryName', 'Senegal')]
[('playerName', 'Mouhamed Sene'), ('teamName', 'New York Knicks'), ('countryName', 'Senegal')]
[('playerName', 'Joakim Noah'), ('teamName', 'New York Knicks'), ('countryName', 'France')]
[('playerName', 'Ronny Turiaf'), ('teamName', 'New York Knicks'), ('countryName', 'France')]
[('playerName', 'Frank Ntilikina'), ('teamName', 'New York Knicks'), ('countryName', 'France')]
[('playerName', 'Kevin Seraphin'), ('teamName', 'New York Knicks'), ('countryName', 'France')]
[('playerName', 'Alexey Shved'), ('teamName', 'New York Knicks'), ('countryName', 'Russia')]
[('playerName', 'Timofey Mozgov'), ('teamName', 'New York Knicks'), ('countryName', 'Russia')]
[('playerName', 'RJ Barrett'), ('teamName', 'New York Knicks'), ('countryName', 'Canada')]
[('playerName', 'Samuel Dalembert'), ('teamName', 'New Yor

In [22]:
# trying to find properties related to continent 
# as an example:

queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # Finding country properties
   ?person wdt:P31 wd:Q5 ;
           wdt:P27 ?country .
   ?country ?p ?o .
   
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1304'), ('pname', 'central bank')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pname', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pname', 'religion')]
[('p', 'http://www.wikidata.org/prop/

In [151]:
#From results: [('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]

queryString = """
SELECT DISTINCT ?playerName ?teamName ?countryName ?continentName
WHERE {
   # all NBA teams
    ?team wdt:P31 wd:Q13393265 ;
          wdt:P118 wd:Q155223 .
   
   #all players played for an NBA team with their nationality
    ?player wdt:P31 wd:Q5 ;
            wdt:P106 wd:Q3665646 ;
            wdt:P54 ?team ;
            wdt:P27 ?country .
   # continent of the player citizenship
    ?country wdt:P30 ?continent .
   
   # get the label
   ?team sc:name ?teamName.
   ?player sc:name ?playerName .
   ?country sc:name ?countryName .
   ?continent sc:name ?continentName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)




Results
[('playerName', 'Ime Udoka'), ('teamName', 'New York Knicks'), ('countryName', 'Nigeria'), ('continentName', 'Africa')]
[('playerName', 'Cheikh Samb'), ('teamName', 'New York Knicks'), ('countryName', 'Senegal'), ('continentName', 'Africa')]
[('playerName', 'Mouhamed Sene'), ('teamName', 'New York Knicks'), ('countryName', 'Senegal'), ('continentName', 'Africa')]
[('playerName', 'Joakim Noah'), ('teamName', 'New York Knicks'), ('countryName', 'France'), ('continentName', 'Europe')]
[('playerName', 'Ronny Turiaf'), ('teamName', 'New York Knicks'), ('countryName', 'France'), ('continentName', 'Europe')]
[('playerName', 'Frank Ntilikina'), ('teamName', 'New York Knicks'), ('countryName', 'France'), ('continentName', 'Europe')]
[('playerName', 'Kevin Seraphin'), ('teamName', 'New York Knicks'), ('countryName', 'France'), ('continentName', 'Europe')]
[('playerName', 'Alexey Shved'), ('teamName', 'New York Knicks'), ('countryName', 'Russia'), ('continentName', 'Europe')]
[('playerNam

Final query for this task

In [88]:

#From results: [('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]

queryString = """
SELECT DISTINCT ?continent ?continentName (COUNT(DISTINCT ?player) as ?players)
WHERE {
   # all NBA teams
    ?team wdt:P31 wd:Q13393265 ;
          wdt:P118 wd:Q155223 .
   
   #all players played for an NBA team with their nationality
    ?player wdt:P31 wd:Q5 ;
            wdt:P106 wd:Q3665646 ;
            wdt:P54 ?team ;
            wdt:P27 ?country .
   # continent of the player citizenship
    ?country wdt:P30 ?continent .
   
   # get the label
   #?team sc:name ?teamName.
   #?player sc:name ?playerName .
   #?country sc:name ?countryName .
   ?continent sc:name ?continentName .
}
GROUP BY (?continent) (?continentName)
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '300')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('players', '3770')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('players', '60')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('players', '73')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentName', 'Eurasia'), ('players', '17')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '37')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '3820')]
7


## Task 8
Identify the BGP for the NBA Defensive Player of the Year Award (see https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award)

In [41]:
#From Task 1: [('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]


queryString = """
SELECT DISTINCT ?award ?awardName
WHERE {
   # awards received by Jordan
    wd:Q41421 wdt:P166 ?award .
   
   # get the label
   ?award sc:name ?awardName .
  
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('award', 'http://www.wikidata.org/entity/Q1061233'), ('awardName', "L'Équipe Champion of Champions")]
[('award', 'http://www.wikidata.org/entity/Q17144'), ('awardName', 'Presidential Medal of Freedom')]
[('award', 'http://www.wikidata.org/entity/Q743122'), ('awardName', 'Associated Press Athlete of the Year')]
[('award', 'http://www.wikidata.org/entity/Q137003'), ('awardName', '50 Greatest Players in NBA History')]
[('award', 'http://www.wikidata.org/entity/Q1418009'), ('awardName', 'NBA All-Rookie Team')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('awardName', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q1641970'), ('awardName', 'John R. Wooden Award')]
[('award', 'http://www.wikidata.org/entity/Q1653207'), ('awardName', 'Sports Illustrated Sportsperson of the Year')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('awardName', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q2404133'), ('awardName',

Final query for this task

In [24]:
queryString = """
SELECT DISTINCT ?award ?awardName
WHERE {
   # awards received by Jordan
    wd:Q41421 wdt:P166 ?award .
   
   # get the label
   ?award sc:name ?awardName .
  
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1061233'), ('awardName', "L'Équipe Champion of Champions")]
[('award', 'http://www.wikidata.org/entity/Q17144'), ('awardName', 'Presidential Medal of Freedom')]
[('award', 'http://www.wikidata.org/entity/Q743122'), ('awardName', 'Associated Press Athlete of the Year')]
[('award', 'http://www.wikidata.org/entity/Q137003'), ('awardName', '50 Greatest Players in NBA History')]
[('award', 'http://www.wikidata.org/entity/Q1418009'), ('awardName', 'NBA All-Rookie Team')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('awardName', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q1641970'), ('awardName', 'John R. Wooden Award')]
[('award', 'http://www.wikidata.org/entity/Q1653207'), ('awardName', 'Sports Illustrated Sportsperson of the Year')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('awardName', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q2404133'), ('awardName',

## Task 9
Consider the players who won the NBA Defensive Player of the Year Award. Return for these players the number of NBA Defensive Player of the Year Award won in their career (the result set must be triples of player IRI, label and #victories)

In [83]:
# From Task 8: [('award', 'http://www.wikidata.org/entity/Q845884'), ('awardName', 'NBA Defensive Player of the Year Award')]

#[('job', 'http://www.wikidata.org/entity/Q3665646'), ('jobName', 'basketball player')]
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]

queryString = """
SELECT DISTINCT ?player ?playerName
WHERE {
   # All basketball players who won NBA Defensive Player of the Year Award
    ?player wdt:P106 wd:Q3665646 ;
            wdt:P166 wd:Q845884 .
            
   
   # get the label
   ?player sc:name ?playerName.
  
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('player', 'http://www.wikidata.org/entity/Q177928'), ('playerName', 'Sidney Moncrief')]
[('player', 'http://www.wikidata.org/entity/Q310968'), ('playerName', 'Alonzo Mourning')]
[('player', 'http://www.wikidata.org/entity/Q971538'), ('playerName', 'Michael Cooper')]
[('player', 'http://www.wikidata.org/entity/Q189240'), ('playerName', 'Kevin Garnett')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('playerName', 'Michael Jordan')]
[('player', 'http://www.wikidata.org/entity/Q273256'), ('playerName', 'Hakeem Olajuwon')]
[('player', 'http://www.wikidata.org/entity/Q211876'), ('playerName', 'David Robinson')]
[('player', 'http://www.wikidata.org/entity/Q361134'), ('playerName', 'Marcus Camby')]
[('player', 'http://www.wikidata.org/entity/Q295322'), ('playerName', 'Metta Sandiford-Artest')]
[('player', 'http://www.wikidata.org/entity/Q926248'), ('playerName', 'Kawhi Leonard')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('playerName', 'Joakim Noah')]
[('player'

In [93]:

queryString = """
SELECT DISTINCT ?player ?playerName (COUNT(wd:Q845884) as ?DPYaward)
WHERE {
   # All basketball players who won NBA Defensive Player of the Year Award
    ?player wdt:P106 wd:Q3665646 ;
            wdt:P166 wd:Q845884 .
            
   
   # get the label
   ?player sc:name ?playerName.
  
   
}
GROUP BY (?player) (?playerName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q41421'), ('playerName', 'Michael Jordan'), ('DPYaward', '1')]
[('player', 'http://www.wikidata.org/entity/Q357757'), ('playerName', 'Mark Eaton'), ('DPYaward', '1')]
[('player', 'http://www.wikidata.org/entity/Q361134'), ('playerName', 'Marcus Camby'), ('DPYaward', '1')]
[('player', 'http://www.wikidata.org/entity/Q201608'), ('playerName', 'Dennis Rodman'), ('DPYaward', '1')]
[('player', 'http://www.wikidata.org/entity/Q3446792'), ('playerName', 'Rudy Gobert'), ('DPYaward', '1')]
[('player', 'http://www.wikidata.org/entity/Q310968'), ('playerName', 'Alonzo Mourning'), ('DPYaward', '1')]
[('player', 'http://www.wikidata.org/entity/Q275889'), ('playerName', 'Marc Gasol'), ('DPYaward', '1')]
[('player', 'http://www.wikidata.org/entity/Q295322'), ('playerName', 'Metta Sandiford-Artest'), ('DPYaward', '1')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('playerName', 'Joakim Noah'), ('DPYaward', '1')]
[('player', 'http://www.wiki

In [109]:
# From Task 8: [('award', 'http://www.wikidata.org/entity/Q845884'), ('awardName', 'NBA Defensive Player of the Year Award')]

queryString = """
SELECT DISTINCT ?p ?pname

WHERE {
   # NBA Defensive Player of the Year Award sub properties
    wd:Q845884 wdt:P1647 ?o .
   
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname .
  
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [94]:

queryString = """
SELECT ?winner ?playerName
WHERE {
   # All basketball players who won NBA Defensive Player of the Year Award
    
    wd:Q845884 wdt:P1346 ?winner.
    

            
   
   # get the label
   ?winner sc:name ?playerName.
   #?p sc:name ?pname .
   #?o sc:name ?oname .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q177928'), ('playerName', 'Sidney Moncrief')]
[('winner', 'http://www.wikidata.org/entity/Q310968'), ('playerName', 'Alonzo Mourning')]
[('winner', 'http://www.wikidata.org/entity/Q971538'), ('playerName', 'Michael Cooper')]
[('winner', 'http://www.wikidata.org/entity/Q189240'), ('playerName', 'Kevin Garnett')]
[('winner', 'http://www.wikidata.org/entity/Q41421'), ('playerName', 'Michael Jordan')]
[('winner', 'http://www.wikidata.org/entity/Q273256'), ('playerName', 'Hakeem Olajuwon')]
[('winner', 'http://www.wikidata.org/entity/Q211876'), ('playerName', 'David Robinson')]
[('winner', 'http://www.wikidata.org/entity/Q361134'), ('playerName', 'Marcus Camby')]
[('winner', 'http://www.wikidata.org/entity/Q295322'), ('playerName', 'Metta Sandiford-Artest')]
[('winner', 'http://www.wikidata.org/entity/Q926248'), ('playerName', 'Kawhi Leonard')]
[('winner', 'http://www.wikidata.org/entity/Q311735'), ('playerName', 'Joakim Noah')]
[('winner'

Final query for this task

In [118]:
"""It seems it is impossible to count the number of awards each player won without "point in time" property and we do not have access to that. no other accessible property exists to distinguish same awards with different dates or at least I could not find that."""

'It seems it is impossible to count the number of awards each player won without "point in time" property and we do not have access to that. no other accessible property exists to distinguish same awards with different dates or at least I could not find that.'